### Modelo de redes neuronales: LiftAdvisor

Con la intención de cumplir con los objetivos planteados, hemos subdividido nuestro modelo de redes neuronales general en dos partes:
1. Modelo de detección de poses: Este modelo funcionará con redes neuronales convolucionales, con el objetivo de captar las imágenes necesarias para detectar la pose de sentadilla, ya sea de frente o de lado, y devolver las coordenadas de las articulaciones de interés.
2. Modelo de clasificación y calificación de técnica: Funciona con una red neuronal común, la cual en base a ciertos criterios (que serán especificados posteriormente), recibe las coordenadas halladas en el modelo anterior, y determina la calidad de la sentadilla en un sistema de calificación del 1 al 10.

Las unidades de trabajo en la clase de Matemáticas Aplicadas II que se ven involucradas en este proyecto son:
1. Unidad 2: Vectores y matrices
2. Unidad 3: Optimización

Nota: En esta dinámica, para el planteamiento matemático primero se explicará el modelo de clasificación y calificación de técnica, ya que es mucho más sencillo y sirve de preámbulo para entender matemáticamente el modelo de detección de poses, que implementa funciones un poco más complejas.

Subdividiremos este proyecto en las siguientes partes:
1. Preprocesamiento y preparación de datos: Divideremos nuestro lote de videos en el grupo de entrenamiento y técnica. La intención es que el modelo de detección de poses devuelva tanto la lista de coordenadas de articulaciones de interés, y se haga un etiquetado automático para poder entrenar y evaluar al segundo modelo.
2. Desarrollo del modelo: Se desarrollará el modelo de clasificación y calificación, explicando a profundiad el algoritmo de forward propagation.
3. Entrenamiento del modelo: Ya que el éxito del modelo depende de su aprendizaje, plantearemos el algoritmo de backpropagation.
4. Evaluación del modelo: En base a nuestro lote de videos y haciendo uso del output del primer modelo, entrenaremos al segundo modelo para que sea capaz de calificar la técnica de una sentadilla de forma correcta.

### 1. Preprocesamiento y preparación de datos

Crearemos un modelo implementando las bibliotecas OpenCV y MediaPipe, con el objetivo de detectar la pose de sentadilla (ya sea de frente o de lado) y obtener las coordenadas de las articulaciones de interés. Posteriormente, haremos un proceso de etiquetado automático para cada criterio, y procesaremos nuestro lote de videos de modo que después pueda ser usado para el entrenamiento y evaluación del segundo modelo de redes neuronales.

pd: Recordar que la explicación matemática de este primer modelo se realizará posteriormente.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [ ]:
# Función para verificar la pose de sentadilla
def es_sentadilla(keypoints):
    # Supongamos que la pose de sentadilla se detecta si la rodilla derecha está por debajo de la cadera
    cadera = keypoints[24]  # La cadera generalmente corresponde al índice 24
    rodilla_derecha = keypoints[26]  # La rodilla derecha generalmente corresponde al índice 26
    
    if rodilla_derecha[1] > cadera[1]:  # Comparar coordenadas y
        return True
    else:
        return False

In [ ]:

# Inicializar Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Crear carpeta para guardar los datos si no existe
if not os.path.exists('squat_data'):
    os.makedirs('squat_data')

# Función para procesar el video y detectar la pose de sentadilla
def detectar_sentadilla(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir imagen de BGR a RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detectar poses
        results = pose.process(image)

        # Convertir imagen de RGB a BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            # Extraer coordenadas de los puntos clave
            keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark])

            # Guardar los puntos clave en un archivo .npy
            np.save(f'squat_data/frame_{frame_count}.npy', keypoints)

            # Verificar si la pose es una sentadilla
            if es_sentadilla(keypoints):
                cv2.putText(image, 'Sentadilla detectada', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Dibujar las poses en la imagen
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Mostrar el video con las poses detectadas
        cv2.imshow('Pose Detection', image)
        frame_count += 1

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Utilizar la función para procesar el video y detectar la pose de sentadilla
video_path = 'ruta/al/video.mp4'
detectar_sentadilla(video_path)